<a href="https://colab.research.google.com/github/AliNajafpour/AI-Face-Identifier/blob/main/TextAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [112]:
import transformers
import os

In [105]:
! git clone https://github.com/AliNajafpour/AI-Face-Identifier.git

Cloning into 'AI-Face-Identifier'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 104 (delta 41), reused 37 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (104/104), 348.61 KiB | 1.41 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [41]:
question_answering_generator = transformers.pipeline('question-answering', model='deepset/roberta-base-squad2', device=0)
text2text_generator = transformers.pipeline("text2text-generation", model="google/flan-t5-base", device=0)
ner_generator = transformers.pipeline("ner", grouped_entities=True)

Device set to use cuda:0
Device set to use cuda:0
No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClass

# Find the name with NER & text2text & question_answering

In [99]:
def extract_name_with_file(file, question):
  with open(file) as f:
    text = f.read()
  result1 = question_answering_generator(context=text, question=question)
  result2 = text2text_generator(f"who is main person in the text?.\n\n {text}")
  result3 = ner_generator(text)
  l = []
  for r in result3:
    if r['entity_group'] == 'PER':
      l.append(r['word'])
  return  result1,  result2,  max(set(l), key=l.count)


In [121]:
name1, name2, name3 = extract_name_with_file('/content/AI-Face-Identifier/test/test_text/text1.txt', 'who is the main person in the texts?')
print('model 1 says: ' + name1['answer'], 'model 2 says: '  + name2[0]['generated_text'], 'model 3 says: ' + name3)

model 1 says: Ayatollah Ali Khamenei model 2 says: President Trump praises Puerto Rico’s response to Hurricane Maria as “an incredible, unsung success model 3 says: Whitehouse


In [107]:
name1, name2, name3 = extract_name_with_file('/content/AI-Face-Identifier/test/test_text/text2.txt', 'who is the main person in the texts?')
print('model 1 says: ' + name1['answer'], 'model 2 says: '  + name2[0]['generated_text'], 'model 3 says: ' + name3)

model 1 says: Schumer model 2 says: Whitehouse says Republicans forced Democrats to choose between supporting a continuing resolution or a government shutdown. model 3 says: Whitehouse


In [108]:
name1, name2, name3 = extract_name_with_file('/content/AI-Face-Identifier/test/test_text/text3.txt', 'who is the main person in the texts?')
print('model 1 says: ' + name1['answer'], 'model 2 says: '  + name2[0]['generated_text'], 'model 3 says: ' + name3)

model 1 says: Ayatollah Ali Khamenei model 2 says: Summary: China to continue retaliation against US tariffs model 3 says: Trump


In [109]:
name1, name2, name3 = extract_name_with_file('/content/AI-Face-Identifier/test/test_text/text4.txt', 'who is the main person in the texts?')
print('model 1 says: ' + name1['answer'], 'model 2 says: '  + name2[0]['generated_text'], 'model 3 says: ' + name3)

model 1 says: President Donald Trump model 2 says: Trump model 3 says: Trump


In [111]:
name1, name2, name3 = extract_name_with_file('/content/AI-Face-Identifier/test/test_text/text5.txt', 'who is the main person in the texts?')
print('model 1 says: ' + name1['answer'], 'model 2 says: '  + name2[0]['generated_text'], 'model 3 says: ' + name3)

model 1 says: Kushner model 2 says: Donald Trump model 3 says: Trump


# Find the name with NER throu all texts

In [148]:
def extract_name_NER(folder_path):
  names = []
  for path in os.listdir(folder_path):
    if path.lower().endswith('.txt'):
        full_path = os.path.join(folder_path, path)
        with open(full_path) as f:
          text = f.read()
        result = ner_generator(text)
        file_names = []
        for r in result:
            if r['entity_group'] == 'PER':
              names.append(r['word'])
  return max(set(names), key=names.count)

In [151]:
extract_name_NER('/content/AI-Face-Identifier/test/test_text')

'Trump'